In [9]:
import os
os.chdir("/Users/dbanh_side/Documents/main_files/GCP/explain-dpm/uploads/script_Modular_Learning/")

import numpy as np
import pandas as pd

import re
import time
import glob

import matplotlib.pyplot as plt
import matplotlib.colors as col
import matplotlib.image as mpimg

import scipy.stats as scp_stat
from scipy.ndimage import gaussian_filter
from scipy.optimize import minimize
import scipy.spatial.distance

from skimage.transform import resize

from sklearn import mixture
from sklearn.metrics import roc_curve, auc

from PIL import Image

import os
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt; 
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.utils import np_utils
from keras.models import Sequential
from keras.applications import VGG16, ResNet50
from keras.applications import imagenet_utils
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import Dense, Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D
import warnings
warnings.filterwarnings('ignore')

In [125]:
def create_features(dataset):
 
    x_scratch = []

    width = 32
    height = 32
    
    for i_id in range(len(dataset)):
        print(i_id)
        image = Image.open(dataset[i_id]).convert("RGB")

        imgheight, imgwidth, _ = np.array(image).shape

        half_imgheight = imgheight%2 + imgheight//2
        half_imgwidth = imgwidth%2 + imgwidth//2

        imgheight = 32
        imgwidth = 32

        for i in range(imgheight//height):
            for j in range(imgwidth//width):

                box = (half_imgwidth - imgwidth + j*width,
                       half_imgheight - imgheight + i*height,
                       half_imgwidth - imgwidth + (j+1)*width,
                       half_imgheight - imgheight + (i+1)*height)

                a = np.array(image.crop(box))
                a = np.expand_dims(a, axis=0)
                x_scratch += [a]
                                
    x = np.vstack(x_scratch)
    return x

In [126]:
roi_files = np.sort(glob.glob("./data/external/extra/ROI_reports/**/*.png",recursive=False))
segments = np.sort(glob.glob("./data/external/extra/ROI_reports/**/*Segments.png",recursive=False))
WT1 = np.sort(glob.glob("./data/external/extra/ROI_reports/**/*WT1.png",recursive=False))

remove_files1 = np.sort(glob.glob("./data/external/extra/ROI_reports/**/*normal3**26**.png",recursive=False))
remove_files2 = np.sort(glob.glob("./data/external/extra/ROI_reports/**/*normal3**27**.png",recursive=False))
remove_files3 = np.sort(glob.glob("./data/external/extra/ROI_reports/**/*normal3**28**.png",recursive=False))

roi_files = np.sort([i for i in roi_files if i not in WT1 and i not in segments and i not in remove_files1 and i not in remove_files2 and i not in remove_files3])
roi_files = roi_files[np.argsort(np.char.lower(np.asarray(roi_files).astype("str")))]

In [127]:
train_data = pd.DataFrame((create_features(roi_files)).reshape((231,-1)))
train_data.to_csv('./data/workflow/TRAIN_pixels_color.csv')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230


In [78]:
train_features_resnet50 = pd.DataFrame(data= train_features)
train_features_resnet50.to_csv('./data/workflow//TRAIN_tile_of_tile_segment.csv')

In [122]:
def create_test_features(dataset):
 
    x_scratch = []

    width = 32
    height = 32
    
    for i_id in range(len(dataset)):
        print(i_id)
        image = Image.open(dataset[i_id]).convert("RGB")

        imgheight, imgwidth, _ = np.array(image).shape

        for i in range(imgheight//height):
            for j in range(imgwidth//width):

                box = (j*width,
                       i*height,
                       (j+1)*width,
                       (i+1)*height)

                a = np.array(image.crop(box))
                a = np.expand_dims(a, axis=0)
                x_scratch += [a]
                                
    x = np.vstack(x_scratch)
    return x
#     features = pre_model.predict(x, batch_size=32)
#     features_flatten = features.reshape((features.shape[0], 2048))
#     return x, features, features_flatten

In [123]:
other_roi_files = np.sort(glob.glob("./data/external/train_test_tiles/*.png",recursive=True))
# test_x, test_features, test_features_flatten = create_test_features(other_roi_files, model)

In [124]:
test_data = pd.DataFrame((create_test_features(other_roi_files)).reshape((1600,-1)))
test_data.to_csv('./data/workflow/TEST_pixels_color.csv')

0


In [118]:
test_data = pd.DataFrame((create_test_features(other_roi_files)).reshape((1600,-1)))
test_data.to_csv('./data/workflow/TEST_pixels.csv')

0
